## Import packages

In [1]:
from opt import get_opts
from config import Config
from models import BasicNeRF, VanillaNeRF, HashNeRF
from model_utils import generate_rays, perform_volume_rendering
from render import pose_spherical

import gin
from pickletools import optimize

import numpy as np
import jax
from jax import numpy as jnp
from jax import jit, pmap, value_and_grad, lax, \
                local_device_count, \
                device_put_replicated, local_devices

from flax.jax_utils import unreplicate
from flax.training import train_state, common_utils
import optax

from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio

## Run on TPU if using Colab (*optional*)

In [2]:
import jax
import os

if 'TPU_NAME' in os.environ:
    import requests
    if 'TPU_DRIVER_MODE' not in globals():
        url = 'http:' + os.environ['TPU_NAME'].split(':')[1] + ':8475/requestversion/tpu_driver_nightly'
        resp = requests.post(url)
        TPU_DRIVER_MODE = 1
    from jax.config import config
    config.FLAGS.jax_xla_backend = "tpu_driver"
    config.FLAGS.jax_backend_target = os.environ['TPU_NAME']
    print("TPU DETECTED!")
    print('Registered TPU:', config.FLAGS.jax_backend_target)


# Detect if Google Colab Notebook has access to TPUs or not
elif "COLAB_TPU_ADDR" in os.environ:
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()


else:
    print('No TPU detected.')


DEVICE_COUNT = len(jax.local_devices())
TPU = DEVICE_COUNT>=1
print("DEVICE_COUNT: ", DEVICE_COUNT)

if TPU:
    print("8 cores of TPU ( Local devices in Jax ):")
    print('\n'.join(map(str,jax.local_devices())))


No TPU detected.
DEVICE_COUNT:  1
8 cores of TPU ( Local devices in Jax ):
TFRT_CPU_0


## Load configurations and data

In [3]:
#gin_config_path = "configs/default.gin"
gin_config_path = "configs/hashnerf.gin"
gin.parse_config_file(gin_config_path)

config = Config()
print(config)

# load data
data = np.load("tiny_nerf_data.npz")
images = data["images"]
poses = data["poses"]
focal = float(data["focal"])

# split data into train and val
_, img_ht, img_wid, _ = images.shape
train_images, train_poses = images[:100], poses[:100]
val_image, val_pose = images[101], poses[101]

Config(near_bound=2.0, far_bound=6.0, num_sample_points=256, epsilon=10000000000.0, apply_positional_encoding=True, positional_encoding_dims=6, positional_encoding_dims_xyz=10, positional_encoding_dims_dir=4, positional_encoding_min_degree=1, positional_encoding_max_degree=10, bbox3D=[[-2.7, -2.7, -2.7], [2.2, 2.2, 2.2]], num_dense_layers=3, dense_layer_width=64, num_dense_layers_dir=4, dense_layer_width_dir=64, skip_layer=0, batch_size=64, lr=0.0005, train_epochs=1000, plot_interval=100)


## Define routines for Training

In [4]:
from hash_encoding import HashEmbedder
from models import VanillaNeRF


def init_model(key, input_pts_shape):
    # create models
    #model = BasicNeRF()
    #model = VanillaNeRF()
    model = HashNeRF()

    # init model
    init_params = jit(model.init)({"params":key}, jnp.ones(input_pts_shape))

    return model, init_params["params"]


def train_step(state, batch, rng):
    """
    Training Step:
    state is comb of model state and optimizer state
    """
    inputs, targets = batch

    # compute loss in a stateless manner to save memory
    def loss_fn(params):
        ray_origins, ray_directions = inputs
        model_fn = lambda x: state.apply_fn({"params": params}, x)
        pred_rgbs, *_ = perform_volume_rendering(model_fn, ray_origins, ray_directions, rng)
        return jnp.mean( (pred_rgbs - targets)**2 )

    # get loss value and gradients    
    train_loss, gradients = value_and_grad(loss_fn)(state.params)

    # get averaged train loss
    train_loss = jnp.mean(train_loss)

    # compute all-reduce mean on gradients over the pmapped axis
    gradients = lax.pmean(gradients, axis_name="batch")

    # update model params & optimizer state
    new_state = state.apply_gradients(grads=gradients)

    # compute PSNR
    train_psnr = -10.0 * jnp.log(train_loss) / jnp.log(10.0)
    return train_loss, train_psnr, new_state

## Generate rays and init model

In [5]:
# create train and validation rays
train_rays = np.stack(list(map(lambda x: generate_rays(img_ht, img_wid, focal, x), train_poses)))
val_rays = generate_rays(img_ht, img_wid, focal, val_pose)

# number of devices
n_devices = local_device_count()
print("Using {} devices to compute".format(n_devices))

# rand number generation
key, rng = jax.random.split(jax.random.PRNGKey(0))

# init the model
model, params = init_model(key, (img_ht*img_wid, 3))

# optimizer
optimizer = optax.adam(learning_rate=config.lr)

# training state
state = train_state.TrainState.create(apply_fn=model.apply, params=params, tx=optimizer)

# transfer arrays in the state to specified devices and form ShardedDeviceArrays
state = device_put_replicated(state, local_devices())

Using 1 devices to compute


KeyboardInterrupt: 

## Define routines for validation

In [ ]:
@jit
def validation_step(state):
    """
    Validation
    """
    model_fn = lambda x: state.apply_fn({"params": state.params}, x)
    ray_origins, ray_directions = val_rays

    pred_rgb, pred_depth, *_ = perform_volume_rendering(model_fn, ray_origins, ray_directions)

    val_loss = jnp.mean( (pred_rgb - val_image)**2 )
    val_psnr = -10.0 * jnp.log(val_loss) / jnp.log(10.0)
    return pred_rgb, pred_depth, val_psnr, val_loss

In [ ]:
def train_and_evaluate(state, train_step_fn, validation_step_fn):
    train_loss_history, train_psnr_history = [], []
    val_loss_history, val_psnr_history = [], []

    for epoch in tqdm(range(config.train_epochs)):
        # shard random number generators
        rng_index, rng_epoch = jax.random.split(jax.random.fold_in(rng, epoch))
        sharded_rngs = common_utils.shard_prng_key(rng_epoch)

        # create training batch 
        train_index = jax.random.randint(rng_index, (n_devices,), minval=0, maxval=len(train_rays))
        train_batch = train_rays[tuple(train_index), ...],\
                      train_images[tuple(train_index), ...]
        
        # perform training step
        train_loss, train_psnr, state = train_step_fn(state, train_batch, sharded_rngs)
        avg_train_loss = np.asarray(np.mean(train_loss))
        avg_train_psnr = np.asarray(np.mean(train_psnr))
        train_loss_history.append(avg_train_loss)
        train_psnr_history.append(avg_train_psnr)
        
        # logging
        print("Train loss @ epoch {}: {}".format(epoch, avg_train_loss))
        print("Train psnr @ epoch {}: {}".format(epoch, avg_train_psnr))

        # perform validation step
        validation_state = unreplicate(state)
        rgb, depth, val_psnr, val_loss = validation_step_fn(validation_state)
        avg_val_loss = np.asarray(np.mean(val_loss))
        avg_val_psnr = np.asarray(np.mean(val_psnr))
        val_loss_history.append(avg_val_loss)
        val_psnr_history.append(avg_val_psnr)
        
        # logging
        print("Val loss @ epoch {}: {}".format(epoch, avg_train_loss))
        print("Val psnr @ epoch {}: {}".format(epoch, avg_train_psnr))

        # plot result at every plot interval
        if epoch % config.plot_interval == 0:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
            ax1.imshow(rgb)
            ax1.set_title("Predicted RGB at epoch: {}".format(epoch))
            ax1.axis('off')
            ax2.imshow(depth)
            ax2.set_title('Predicted RGB at epoch: {}'.format(epoch))
            ax2.axis('off')
            plt.show()
    
    inference_state = unreplicate(state)
    history = {
        'train_loss': train_loss_history,
        'train_psnr': train_psnr_history,
        'val_loss': val_loss_history,
        'val_psnr': val_psnr_history
    }
    return state, inference_state, history

## Start training

In [ ]:

# perform training now
# Apply the transform jax.pmap on the train_step to parallelize it on XLA devices
# While vmap vectorizes a function by adding a batch dimension to every primitive operation 
# in the function, pmap replicates the function and executes each replica 
# on its own XLA device in parallel.
parallelized_train_step = pmap(train_step, axis_name="batch")
state, inference_state, history = train_and_evaluate(state, parallelized_train_step, validation_step)


  0%|          | 0/1000 [00:11<?, ?it/s]


KeyboardInterrupt: 

## Render the scene

In [ ]:
@jit
def get_renderings(rays):
    model_fn = lambda x: inference_state.apply_fn(
        {"params": inference_state.params}, x
    )

    ray_origins, ray_directions = rays
    rgb, depth, acc, disparity, opacities = perform_volume_rendering(
        model_fn, ray_origins, ray_directions
    )
    rgb = (255 * jnp.clip(rgb, 0, 1)).astype(jnp.uint8)
    return rgb, depth, acc, disparity, opacities


# Create a 360 degree video of the 3D scene
def get_frames():
    video_angle = jnp.linspace(0.0, 360.0, 120, endpoint=False)
    camera_to_world_transform = map(lambda th: pose_spherical(th, -30.0, 4.0), video_angle)
    rays = np.stack(list(map(
        lambda x: generate_rays(
            img_ht, img_wid, focal, x[:3, :4]
        ), camera_to_world_transform
    )))
    rgb_frames, depth_frames, acc_maps, disparity_maps, opacities = lax.map(get_renderings, rays)
    rgb_frames = np.asarray(rgb_frames)
    depth_frames = np.asarray(depth_frames)
    acc_maps = np.asarray(acc_maps * 255.)
    disparity_maps = np.asarray(disparity_maps * 255.)
    return rgb_frames, depth_frames, acc_maps, disparity_maps

In [ ]:
# render the scene
rgb_frames, depth_frames, acc_maps, disparity_maps = get_frames()

## Export videos

In [ ]:
# output videos
imageio.mimwrite("rgb_video.mp4", tuple(rgb_frames), fps=30, quality=7)
imageio.mimwrite("depth_video.mp4", tuple(depth_frames), fps=30, quality=7)
imageio.mimwrite("acc_video.mp4", tuple(acc_maps), fps=30, quality=7)
imageio.mimwrite("disparity_video.mp4", tuple(disparity_maps), fps=30, quality=7)